#### Import Necessary Library

In [1]:
import os
from dotenv import load_dotenv

from langchain_groq import ChatGroq   # Or ChatOpenAI if you prefer OpenAI
from langchain.chains import create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.documents import Document
import streamlit as st
from langchain_core.prompts import ChatPromptTemplate
#from langchain_core.output_parsers import StrOutputParser
#from langchain_community.llms import Ollama

In [ ]:
## Langsmith Tracking
#os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
#os.environ["LANGCHAIN_TRACING_V2"]="true"
#os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [2]:
# set embedding tech
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings

g:\Gen AI-Krish Naik\5. Personal Gen AI Project\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [3]:
# load from disk -chromadb
from langchain_chroma import Chroma
new_db = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)

In [4]:
from langchain_community.vectorstores import FAISS
# load from disk
new_db = FAISS.load_local(INDEX_PATH,embeddings,allow_dangerous_deserialization=True)

In [4]:
query="Do you support Facebook Messenger?"
docs=new_db.similarity_search(query)
docs

[Document(id='697d5e50-22cc-44d9-92ba-be58eae49886', metadata={}, page_content="Q: How do I reset my password?\nA: Click on 'Forgot Password' on the login page and follow the steps.\nQ: What should I do if my laptop doesn’t turn on?\nA: Please check the power connection and try holding the power button for 10 seconds.\nQ: My payment failed but money deducted.\nA: Please provide your transaction ID. We will check and process a refund if applicable.")]

In [5]:
# retriever object to query similar documents
retriever = new_db.as_retriever(search_type="similarity", search_kwargs={"k": 3})
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x0000021220A719D0>, search_kwargs={'k': 3})

In [ ]:
# %% 2. Define LLM

In [6]:
import os
from dotenv import load_dotenv
import streamlit as st
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [7]:
load_dotenv()
## load the GROQ API Key
os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")
os.environ['GROQ_API_KEY']=os.getenv("GROQ_API_KEY")

groq_api_key=os.getenv("GROQ_API_KEY")

llm=ChatGroq(groq_api_key=groq_api_key,model_name="llama-3.1-8b-instant")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000021224300BC0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000021224534560>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [8]:
## Prompt Template
qa_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a polite and user-friendly support assistant.\n"
     "Use the following KB context to answer the user question clearly:\n\n"
     "Context:\n{context}\n\n"
     "Guidelines:\n"
     "- Always explain in simple and helpful language.\n"
     "- If the KB does not have enough info, say: 'I’m not sure about that. Please contact support.'"),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}")
])

In [9]:
## Document chain for creation-custom prompt for answering questions with context.

document_chains =  create_stuff_documents_chain(llm,qa_prompt)
document_chains

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['chat_history', 'context', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[l

In [12]:
from langchain_core.documents import Document
document_chains.invoke({
    "input":"Can you integrate with Salesforce?",
    "context":[Document(page_content="ID  \nScenario  Customer  Query  Agent  Response  Category  \n1  Tier  1  Support  \nCan  I  change  my  shipping  address?  \nYes,  you  can  change  your  shipping  address  from  your  account  settings  before  the  order  is  shipped.  \nTier  1  Support  &  Issue  Triage  \n2  Lead  Qualification  \nCan  you  integrate  with  Salesforce?  \nYes,  our  platform  integrates  seamlessly  with  Salesforce  and  other  CRM  tools.  \nLead  Qualification  &  Pre-sales  \n3  Proactive  Support  \nHow  do  I  download  my  invoice?  \nYou  can  download  your  invoice  from  the  Billing  section  in  your  account.  \nProactive  &  Outbound  Support  \n4  Internal  Knowledge  \nWhere  do  I  find  onboarding  guides?  \nOnboarding  guides  are  available  on  the  HR  portal  â†’  New  Employee  section.  \nInternal  Knowledge  Management  \n5  Multi-channel  Support  \nDo  you  support  Facebook  Messenger?  \nYes,  our  agents  are  available  on  Facebook  Messenger  during  business  hours.' ")],
    "chat_history": []
})

'According to our knowledge base, we can indeed integrate with Salesforce. Our platform integrates seamlessly with Salesforce and other CRM tools, which should make it easy to connect your systems and streamline your workflow.'

In [13]:
### Input--->Retriever--->vectorstoredb
new_db

In [14]:
# %% 3. Conversational Retriever Setup
retriever = new_db.as_retriever()

from langchain.chains import create_retrieval_chain

Rag_chains = create_retrieval_chain(retriever,document_chains)


In [15]:
Rag_chains

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x0000021220A719D0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['chat_history', 'context', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.A

In [17]:
## Get the response form the LLM
response = Rag_chains.invoke({ "input": "Who manages payroll queries?",
    "chat_history": [], 
})
response['answer']

"I'm not sure about that. Please contact support."

### User and AI Chat History


In [18]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# Prompt template for conversation

retriever_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. "
    "Rewrite the user question into a standalone query using chat history if needed."),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}")
])

In [19]:
# History-aware retriever (rephrases user queries based on conversation)
history_aware_retriever = create_history_aware_retriever( llm, retriever, retriever_prompt )
history_aware_retriever

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x0000021220A719D0>, search_kwargs={}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessag

In [20]:
# retriver_chain
retrieval_chain = create_retrieval_chain(history_aware_retriever, document_chains)

#### chat conversation history store

In [21]:
# manual chat history store
from langchain_core.messages import AIMessage,HumanMessage
chat_history=[]
question="Can I chat on WhatsApp?"
response1=retrieval_chain.invoke({"input":question,"chat_history":chat_history})

chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=response1["answer"])
    ]
)

question2="How do I update my billing information?"
response2=retrieval_chain.invoke({"input":question,"chat_history":chat_history})
print(response2['answer'])



Unfortunately, I don't have any information about our WhatsApp chat service. I’m not sure about that. Please contact support for more information.


In [22]:
chat_history

[HumanMessage(content='Can I chat on WhatsApp?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="I'm an assistant for our official support channel. However, I don't have information on WhatsApp chat availability in our KB context. I’m not sure about that. Please contact support.", additional_kwargs={}, response_metadata={})]

In [23]:
# automatically store all chat history 
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# Wrap your RAG chain with history support
conversational_rag_chain = RunnableWithMessageHistory(
    retrieval_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [48]:
def ask_question3(question: str):
    response = conversational_rag_chain.invoke(
        {"input": question},
        config={"configurable": {"session_id": "default"}}
    )
    return response["answer"]


In [49]:
print(ask_question3("How do I update my billing information?"))

Updating your billing information is a relatively straightforward process. Here's what you can do:

1. Log in to your account.
2. Go to the 'Account' or 'Profile' section.
3. Look for the 'Billing Information' or 'Payment Details' option and click on it.
4. Update your billing information, such as your address, card details, or bank information.
5. Save your changes.

If you're having trouble finding the 'Account' or 'Profile' section, or if you're not sure about the specific steps, I can suggest trying the website's help section or FAQ for more detailed instructions. If you're still unsure, please contact our support team and we'll be happy to help.


In [50]:
# User 1 asks questions
print(ask_question3("How do I update my billing information?"))


# Check history for user1
history1 = get_session_history("default")

print(ask_question3("My product needs an update"))
for msg in history1.messages:
    print(f"{msg.type.upper()}: {msg.content}")


To update your billing information, follow these steps:

1. Log in to your account.
2. Go to the 'Account' or 'Profile' section.
3. Look for the 'Billing Information' or 'Payment Details' option and click on it.
4. Update your billing information, such as your address, card details, or bank information.
5. Save your changes.

If you're having trouble finding the 'Account' or 'Profile' section, you can also try clicking on the 'Settings' or 'Edit Profile' option in your account dashboard.

Additionally, you can also try searching for "update billing information" in the search bar on the website or checking the website's help section for more detailed instructions. If you're still unsure, please contact our support team and we'll be happy to help.
To update your product, I'm not sure about the exact steps. However, I can suggest a few options:

1. Check the product's manual or the manufacturer's website for instructions on how to update the product.
2. Look for a 'Settings' or 'Update' o

In [51]:
# User 1 asks questions
print(ask_question3("Can I change my  shipping address?"))


Yes, you can change your shipping address. To do so:

1. Log in to your account.
2. Go to the 'Account' or 'Profile' section.
3. Look for the 'Address Book' or 'Shipping Addresses' option and click on it.
4. Edit the shipping address you want to update.
5. Save your changes.

If you're having trouble finding the 'Account' or 'Profile' section, you can also try clicking on the 'Settings' or 'Edit Profile' option in your account dashboard.

If you're trying to change your shipping address for an upcoming order, you might want to check with our support team to see if it's possible to make changes before the order ships.
